# 1 basics
## 1.1 Tokenization

    intro_to_tokenization()
    tokenization_examples()
    character_tokenizer()
    byte_tokenizer()
    word_tokenizer()
    bpe_tokenizer()

### 什么是Tokenization？
原始文本通常用 Unicode 字符串表示。语言模型会对一系列**词元**（tokens）（通常用整数索引表示）放置一个概率分布。因此，我们需要一种将字符串**编码**（encode）成词元的过程，同时需要一种将词元**解码**（decode）回字符串的过程。Tokenizer（分词器）就是一个实现了编码和解码方法的类。**词表大小**（vocabulary size）就是可能的词元（整数）总数。

为了直观了解分词器是如何工作的，可以试用这个[交互式网站](https://tiktokenizer.vercel.app/?encoder=gpt2)。

观察到的现象：
1. 一个单词及其前面的空格会被当作同一个 token（例如 " world"）。
2. 处在句首的单词和处在句中间的同一单词会被不同地表示（例如 "hello hello"）。
3. 数字会被分成每几位一个 token。

以下是 OpenAI 的 GPT-2 分词器（tiktoken）的实际运行示例：

python
复制
编辑
tokenizer = get_gpt2_tokenizer()
string = "Hello, 🌍! 你好!"  # @inspect string

# 检查 encode() 和 decode() 是否能够无损往返
indices = tokenizer.encode(string)  # @inspect indices
reconstructed_string = tokenizer.decode(indices)  # @inspect reconstructed_string
assert string == reconstructed_string

compression_ratio = get_compression_ratio(string, indices)  # @inspect compression_ratio
这段意思是在演示如何用 GPT-2 分词器将字符串编码成 token，再解码回来，并验证它们是否完全一致，同时计算压缩比。

### 基于字符的分词器

### 基于字节的分词器

### 基于单词的分词器

### 基于BPE的分词器

